In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('add_community.csv')
df.head()

df = df[(df['BoroCD'] == 307) | (df['BoroCD'] == 310) | (df['BoroCD'] == 311) | (df['BoroCD'] == 312) | (df['BoroCD'] == 314) | (df['BoroCD'] == 315) | (df['BoroCD'] == 318)]

In [ ]:
df.shape

df.to_csv('brooklyn_AD.csv')

In [ ]:
df.columns

In [ ]:
df = df[['OFNS_DESC','hour','weekday','BoroCD']]
df.head()

In [ ]:
df.columns = [['crime','hour','weekday','community']]
df.head()

In [ ]:
df.shape

In [ ]:
bro = pd.read_csv('Brooklyn_cluster.csv')
del bro[bro.columns[0]]
bro = bro[['community','labels']]
bro = bro[bro['labels'] == 1]
bro = bro.reset_index(drop=True)
bro

In [ ]:
df = df[(df['community'] == 307) | (df['community'] == 310) | (df['community'] == 311) | (df['community'] == 312) | (df['community'] == 314) | (df['community'] == 315) | (df['community'] == 318)]

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
np.unique(df['community'])

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
len(df.crime.unique())

In [ ]:
filter = df.groupby('crime').count()
filter = filter[filter['hour'] > 200]
filter.head()

In [ ]:
len(filter.index.unique())

In [ ]:
df = df[df['crime'].isin(filter.index.unique())]

In [ ]:
df = df.reset_index(drop=True)

## Calculate percent of hour

In [ ]:
df1 = df[['crime','hour']]

In [ ]:
data1 = df1[df1['crime'] == df1['crime'].unique()[0]].groupby('hour').count().T
data1.index = range(0,1)
data1.head()

In [ ]:
for i in range(1,len(df1['crime'].unique())):
    DATA = df1[df1['crime'] == df1['crime'].unique()[i]].groupby('hour').count().T
    DATA.index = range(i,i+1)
    data1 = pd.concat([data1,DATA])

In [ ]:
data1.index = df1['crime'].unique()

In [ ]:
data1.head()

In [ ]:
data1.shape

In [ ]:
data1['sum'] = data1[data1.columns[0]]

for i in range(20):
    data1.iloc[i,-1] = data1.iloc[i,:-1].sum()
    
for i in range(20):
    for j in range(24):
        data1.iloc[i,j] = data1.iloc[i,j] / data1.iloc[i,-1]       
        
data1 = data1.iloc[:,:-1]

data1.columns = 'hour_' + data1.columns.map(str)

hour = data1.copy()

hour.head()

In [ ]:
hour = hour.fillna(0)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
x = hour[[x for x in hour.columns if 'hour' in x]]
score = {}
for i in range(2,10):
    km = KMeans(random_state=999,n_clusters=i)
    res=km.fit(x)
    labels = res.labels_
    score[i] = silhouette_score(x, labels)
    print("For n_clusters = {},".format(i)+" the silhouette_score is : {}".format(score[i]))
print('The choosen number of clusters is {}'.format([n for n in score.keys() if score[n]==max(score.values())][0]))

In [ ]:
from sklearn.cluster import KMeans
km = KMeans(random_state=999,n_clusters=2)
res=km.fit(hour[[x for x in hour.columns if 'hour' in x]])

print 'Cluster assignments:',res.labels_
print '\nCluster centers:\n',res.cluster_centers_
print '\nSum of squared errors:',res.inertia_

In [ ]:
crime = hour[[x for x in hour.columns if 'hour' in x]].copy()
crime['labels'] = res.labels_

In [ ]:
hr_mean = pd.DataFrame({'hour_{}'.format(x):0 for x in range(0,24)},index=[0,1])
hr_mean.columns = sorted(hr_mean.columns,key=lambda x: int(x[5:]))

In [ ]:
for i in range(0,2):
    hr_mean.iloc[i] = crime[crime['labels']==i].mean(0)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
hr_mean.T.plot(figsize=[20,10],xticks=range(24),title='mean values of each cluster for each hour')
plt.show()

In [ ]:
crime.head()

In [ ]:
cluster0 = crime[crime['labels'] == 0]

In [ ]:
cluster0['labels'] = 2

In [ ]:
cluster0

In [ ]:
cluster1 = crime[crime['labels'] == 1].iloc[:,:-1]

In [ ]:
from sklearn.metrics import silhouette_score
x = cluster1
score = {}
for i in range(2,10):
    km = KMeans(random_state=999,n_clusters=i)
    res=km.fit(x)
    labels = res.labels_
    score[i] = silhouette_score(x, labels)
    print("For n_clusters = {},".format(i)+" the silhouette_score is : {}".format(score[i]))
print('The choosen number of clusters is {}'.format([n for n in score.keys() if score[n]==max(score.values())][0]))

In [ ]:
from sklearn.cluster import KMeans
km = KMeans(random_state=999,n_clusters=2)
res=km.fit(x)

print 'Cluster assignments:',res.labels_
print '\nCluster centers:\n',res.cluster_centers_
print '\nSum of squared errors:',res.inertia_

In [ ]:
cluster1['labels'] = res.labels_
cluster1.head()

In [ ]:
cluster1[cluster1['labels'] == 0]

In [ ]:
cluster1[cluster1['labels'] == 1]

In [ ]:
hr_mean = pd.DataFrame({'hour_{}'.format(x):0 for x in range(0,24)},index=[0,1])
hr_mean.columns = sorted(hr_mean.columns,key=lambda x: int(x[5:]))

In [ ]:
for i in range(0,2):
    hr_mean.iloc[i] = cluster1[cluster1['labels']==i].mean(0)

In [ ]:
from matplotlib import pyplot as plt
hr_mean.T.plot(figsize=[20,10],xticks=range(24),title='mean values of each cluster for each hour')
plt.show()

In [ ]:
cluster = pd.concat([cluster1,cluster0])

In [ ]:
hr_mean = pd.DataFrame({'hour_{}'.format(x):0 for x in range(0,24)},index=[0,1,2])
hr_mean.columns = sorted(hr_mean.columns,key=lambda x: int(x[5:]))

In [ ]:
for i in range(0,3):
    hr_mean.iloc[i] = cluster[cluster['labels']==i].mean(0)

In [ ]:
hr_mean

In [ ]:
from matplotlib import pyplot as plt
hr_mean.T.plot(figsize=[20,10],xticks=range(24),title='mean values of each cluster for each hour')
plt.show()

## Calculate percent of weekday

In [ ]:
df.shape

In [ ]:
df1 = df[['crime','weekday']]
df1.head()

In [ ]:
merge =pd.DataFrame([['SUNDAY',7],['MONDAY',1],['TUESDAY',2],['WEDNESDAY',3],['THURSDAY',4],['FRIDAY',5],['SATURDAY',6]],columns=['weekday', 'day'])
merge.day = merge.day.astype(int)
merge.day = merge.day.astype(str)
merge.head()

In [ ]:
df1 = pd.merge(merge,df1,on='weekday')
df1 = df1[['crime','day']]
df1.head()

In [ ]:
data1 = df1[df1['crime'] == df1['crime'].unique()[0]].groupby('day').count().T
data1.index = range(0,1)
data1 = data1[['7','1','2','3','4','5','6']]
data1.head()

In [ ]:
for i in range(1,len(df1['crime'].unique())):
    DATA = df1[df1['crime'] == df1['crime'].unique()[i]].groupby('day').count().T
    DATA.index = range(i,i+1)
    data1 = pd.concat([data1,DATA])

In [ ]:
data1.index = df1['crime'].unique()

In [ ]:
data1

In [ ]:
data1['sum'] = data1[data1.columns[0]]

In [ ]:
for i in range(20):
    data1.iloc[i,-1] = data1.iloc[i,:-1].sum()
data1

In [ ]:
for i in data1.columns[:-1]:
    data1[i] = data1[i] / data1['sum']
data1

In [ ]:
data1 = data1.iloc[:,:-1]

weekday = data1.copy()

weekday.columns = [['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']]

weekday.head()

In [ ]:
weekday = weekday.fillna(0)

In [ ]:
from sklearn.metrics import silhouette_score
x = weekday
score = {}
for i in range(2,10):
    km = KMeans(random_state=999,n_clusters=i)
    res=km.fit(x)
    labels = res.labels_
    score[i] = silhouette_score(x, labels)
    print("For n_clusters = {},".format(i)+" the silhouette_score is : {}".format(score[i]))
print('The choosen number of clusters is {}'.format([n for n in score.keys() if score[n]==max(score.values())][0]))

In [ ]:
km = KMeans(random_state=999,n_clusters=2)
res=km.fit(weekday)

print 'Cluster assignments:',res.labels_
print '\nCluster centers:\n',res.cluster_centers_
print '\nSum of squared errors:',res.inertia_

In [ ]:
weekday['labels'] = res.labels_
weekday.head()

In [ ]:
weekday[weekday['labels'] == 1] 

In [ ]:
day_mean = pd.DataFrame({x:0 for x in range(7)},index=['cluster0','cluster1'])
day_mean.columns = weekday.columns[:-1]
for i in range(2):
     day_mean.iloc[i] = weekday[weekday['labels']==i].iloc[:,:-1].mean()
day_mean.head()

In [ ]:
day_mean.T.plot(figsize=[15,8],title='different day-of-week trends for 5 clusters')

In [ ]:
plt.show()

In [ ]:
cluster0 = weekday[weekday['labels'] == 1]

In [ ]:
cluster0['labels'] = 2

In [ ]:
cluster0.index = ['cluster2']

In [ ]:
cluster0 = cluster0.iloc[:,:-1]
cluster0

In [ ]:
cluster1 = weekday[weekday['labels'] == 0].iloc[:,:-1]

In [ ]:
from sklearn.metrics import silhouette_score
x = cluster1
score = {}
for i in range(2,10):
    km = KMeans(random_state=999,n_clusters=i)
    res=km.fit(x)
    labels = res.labels_
    score[i] = silhouette_score(x, labels)
    print("For n_clusters = {},".format(i)+" the silhouette_score is : {}".format(score[i]))
print('The choosen number of clusters is {}'.format([n for n in score.keys() if score[n]==max(score.values())][0]))

In [ ]:
from sklearn.cluster import KMeans
km = KMeans(random_state=999,n_clusters=2)
res=km.fit(x)

print 'Cluster assignments:',res.labels_
print '\nCluster centers:\n',res.cluster_centers_
print '\nSum of squared errors:',res.inertia_

In [ ]:
cluster1['labels'] = res.labels_
cluster1.head()

In [ ]:
cluster1[cluster1['labels'] == 0]

In [ ]:
cluster1[cluster1['labels'] == 1]

In [ ]:
day_mean = pd.DataFrame({x:0 for x in range(7)},index=['cluster0','cluster1'])
day_mean.columns = cluster1.columns[:-1]
for i in range(2):
     day_mean.iloc[i] = cluster1[cluster1['labels']==i].iloc[:,:-1].mean()
day_mean.head()

In [ ]:
day_mean.T.plot(figsize=[15,8],title='different day-of-week trends for 2 clusters')
plt.show()

In [ ]:
day_mean = pd.concat([day_mean,cluster0])
day_mean

In [ ]:
day_mean.T.plot(figsize=[15,8],title='different day-of-week trends for 3 clusters')
plt.show()